# Bab 12: Model Kustom dan Pelatihan dengan TensorFlow

Bab ini akan menyelam lebih dalam ke dalam API tingkat rendah TensorFlow untuk memungkinkan kustomisasi yang lebih mendalam pada model dan proses pelatihan. Kita akan belajar cara membuat fungsi *loss* kustom, metrik, lapisan, dan bahkan loop pelatihan kustom, serta memanfaatkan fitur *graph* otomatis TensorFlow untuk performa yang lebih baik.

## 1. Tur Singkat TensorFlow

TensorFlow adalah pustaka yang kuat untuk komputasi numerik, dioptimalkan untuk *Machine Learning* skala besar.  Meskipun `tf.keras` adalah API tingkat tinggi yang sering kita gunakan, memahami API tingkat rendahnya memberikan fleksibilitas tambahan.


### 1.1. Arsitektur TensorFlow

TensorFlow memiliki arsitektur yang dirancang untuk performa dan skalabilitas:
* **Inti:** Sangat mirip dengan NumPy, tetapi dengan dukungan GPU.
* **Komputasi Terdistribusi:** Mendukung komputasi yang tersebar di banyak perangkat dan *server*.
* **Kompiler Just-In-Time (JIT):** Mengoptimalkan komputasi untuk kecepatan dan penggunaan memori dengan mengekstrak *computation graph* dari fungsi Python.
* ***Computation Graphs* yang Portabel:** *Graph* dapat diekspor untuk dijalankan di berbagai lingkungan (misalnya, Python di Linux, Java di Android).
* ***Autodiff* dan *Optimizers*:** Mengimplementasikan *autodiff* dan menyediakan *optimizer* canggih seperti RMSProp dan Nadam.

Pada tingkat terendah, setiap operasi TensorFlow (disebut *op*) diimplementasikan dalam kode C++ yang sangat efisien.  Banyak operasi memiliki implementasi ganda yang disebut *kernel*, masing-masing didedikasikan untuk jenis perangkat tertentu (CPU, GPU, TPU).

### 1.2. Ekosistem TensorFlow

TensorFlow lebih dari sekadar pustaka; ia adalah bagian dari ekosistem yang luas:
* **TensorBoard:** Alat visualisasi interaktif untuk kurva pembelajaran, *graph* komputasi, statistik pelatihan, dan lainnya.
* **TensorFlow Extended (TFX):** Kumpulan pustaka untuk membuat proyek TensorFlow siap produksi, termasuk validasi data, *preprocessing*, analisis model, dan penyajian.
* **TensorFlow Hub:** Untuk mengunduh dan menggunakan kembali jaringan saraf yang sudah dilatih (*pretrained*).
* **TensorFlow Model Garden:** Berisi banyak arsitektur jaringan saraf, beberapa di antaranya sudah dilatih.
* **TensorFlow Lite:** Untuk menyebarkan model ke perangkat seluler dan *embedded*.
* **TensorFlow.js:** Implementasi JavaScript untuk menjalankan model langsung di *browser* web.

## 2. Menggunakan TensorFlow seperti NumPy

API TensorFlow berpusat pada *tensor*, yang mengalir dari satu operasi ke operasi lainnya.  *Tensor* sangat mirip dengan `NumPy ndarray`: biasanya merupakan array multidimensi, tetapi juga dapat menampung *scalar* (nilai tunggal).

### 2.1. Tensor dan Operasi

Anda dapat membuat *tensor* dengan `tf.constant()`.

In [1]:
import tensorflow as tf
import numpy as np

# Membuat tensor matriks
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t)

# Membuat tensor skalar
s = tf.constant(42)
print(s)

# Mengakses shape dan dtype
print(f"Shape dari t: {t.shape}")
print(f"Dtype dari t: {t.dtype}")

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(42, shape=(), dtype=int32)
Shape dari t: (2, 3)
Dtype dari t: <dtype: 'float32'>


Pengindeksan (*indexing*) bekerja mirip seperti NumPy:

In [2]:
# Pengindeksan baris dan kolom
print(t[:, 1:])

# Pengindeksan elipsis dan dimensi baru
print(t[..., 1, tf.newaxis])

tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2.]
 [5.]], shape=(2, 1), dtype=float32)


Berbagai operasi *tensor* juga tersedia:

In [3]:
# Penjumlahan elemen-demi-elemen
print(t + 10)

# Pangkat elemen-demi-elemen
print(tf.square(t))

# Perkalian matriks (dot product)
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


### 2.2. Tensor dan NumPy

*Tensor* dan NumPy bekerja sama dengan baik: Anda dapat membuat *tensor* dari array NumPy, dan sebaliknya.  Anda bahkan dapat menerapkan operasi TensorFlow ke array NumPy dan operasi NumPy ke *tensor*.

In [4]:
a = np.array([2., 4., 5.])

# Membuat tensor dari array NumPy
tf_from_np = tf.constant(a)
print(tf_from_np)

# Mengubah tensor menjadi array NumPy
np_from_tf = t.numpy() # atau np.array(t)
print(np_from_tf)

# Menerapkan operasi TensorFlow pada array NumPy
tf_op_on_np = tf.square(a)
print(tf_op_on_np)

# Menerapkan operasi NumPy pada tensor TensorFlow
np_op_on_tf = np.square(t)
print(np_op_on_tf)

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
[[1. 2. 3.]
 [4. 5. 6.]]
tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64)
[[ 1.  4.  9.]
 [16. 25. 36.]]


**Catatan Penting:** NumPy menggunakan presisi 64-bit secara *default*, sedangkan TensorFlow menggunakan 32-bit. Disarankan untuk mengatur `dtype=tf.float32` saat membuat *tensor* dari array NumPy untuk konsistensi dan performa.

### 2.3. Konversi Tipe

TensorFlow tidak melakukan konversi tipe secara otomatis untuk menghindari dampak performa yang signifikan dan tidak disadari.  Ini akan memunculkan *exception* jika Anda mencoba melakukan operasi pada *tensor* dengan tipe yang tidak kompatibel.  Anda dapat menggunakan `tf.cast()` jika perlu mengonversi tipe.

In [5]:
# Contoh error karena tipe tidak kompatibel (akan memunculkan error jika dijalankan)
# tf.constant(2.) + tf.constant(40)

# Contoh konversi tipe
t2 = tf.constant(40., dtype=tf.float64)
converted_sum = tf.constant(2.0) + tf.cast(t2, tf.float32)
print(converted_sum)

tf.Tensor(42.0, shape=(), dtype=float32)


### 2.4. Variabel

Nilai `tf.Tensor` yang telah kita lihat sejauh ini bersifat *immutable* (tidak dapat diubah).  Ini berarti kita tidak bisa menggunakan *tensor* biasa untuk mengimplementasikan bobot dalam jaringan saraf, karena bobot perlu disesuaikan oleh *backpropagation*.  Untuk itu, kita memerlukan `tf.Variable`.

In [6]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)

# Modifikasi nilai variabel di tempat (in-place)
v.assign(2 * v)
print(v)

v[0, 1].assign(42)
print(v)

v[:, 2].assign([0., 1.])
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>


Meskipun `tf.Variable` bertindak mirip dengan `tf.Tensor`, dan mendukung operasi yang sama, ia juga dapat dimodifikasi di tempat menggunakan metode `assign()` (atau `assign_add()` atau `assign_sub()`).

### 2.5. Struktur Data Lainnya

TensorFlow mendukung beberapa struktur data lainnya:
* ***Sparse tensors* (`tf.SparseTensor`):** Merepresentasikan *tensor* yang sebagian besar berisi nol secara efisien.
* ***Tensor arrays* (`tf.TensorArray`):** Daftar *tensor*. Ukurannya tetap secara *default* tetapi dapat dibuat dinamis.
* ***Ragged tensors* (`tf.RaggedTensor`):** Merepresentasikan daftar statis dari daftar *tensor* di mana setiap *tensor* memiliki *shape* dan tipe data yang sama, tetapi panjang *slice*nya bisa berbeda.
* ***String tensors*:** *Tensor* biasa dengan `dtype=tf.string`. Merepresentasikan *byte strings* (bukan *Unicode strings*).
* **Sets:** Merepresentasikan *set* sebagai *tensor* biasa (atau *sparse tensor*).
* **Queues:** Menyimpan *tensor* di banyak *step*.

## 3. Mengkustomisasi Model dan Algoritma Pelatihan

Bagian ini membahas cara mengimplementasikan komponen kustom untuk model TensorFlow Anda.

### 3.1. Fungsi *Loss* Kustom

Misalnya Anda ingin melatih model regresi, tetapi *training set* Anda sedikit *noisy*.  Fungsi *loss* *mean squared error* (MSE) mungkin terlalu banyak menghukum *error* besar.  *Mean absolute error* (MAE) tidak terlalu sensitif terhadap *outlier*, tetapi pelatihan bisa lambat.  *Huber loss* (diperkenalkan dalam Bab 10) adalah alternatif yang baik.

Anda dapat membuat fungsi *loss* kustom dengan membuat fungsi Python yang mengambil *label* (`y_true`) dan prediksi (`y_pred`) sebagai argumen, dan mengembalikan *tensor* yang berisi satu *loss* per *instance*.

In [7]:
# Contoh data fiktif
X_train_scaled = np.random.rand(100, 10).astype(np.float32)
y_train = np.random.rand(100, 1).astype(np.float32)
X_valid = np.random.rand(20, 10).astype(np.float32)
y_valid = np.random.rand(20, 1).astype(np.float32)

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=[10]),
    tf.keras.layers.Dense(1)
])

model.compile(loss=huber_fn, optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2, validation_data=(X_valid, y_valid))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 184ms/step - loss: 0.0848 - val_loss: 0.0682
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0804 - val_loss: 0.0621


Untuk performa yang lebih baik, gunakan implementasi *vectorized* dan hanya operasi TensorFlow.

### 3.2. Menyimpan dan Memuat Model yang Mengandung Komponen Kustom

Saat menyimpan model yang berisi fungsi *loss* kustom, Keras menyimpan nama fungsi tersebut.  Saat memuatnya, Anda perlu menyediakan *dictionary* yang memetakan nama fungsi ke fungsi yang sebenarnya.

In [8]:
# Asumsikan model sudah dilatih dan disimpan sebelumnya
# model.save("my_model_with_a_custom_loss.h5")

# Memuat model dengan fungsi loss kustom
# model = tf.keras.models.load_model("my_model_with_a_custom_loss.h5",
#                                    custom_objects={"huber_fn": huber_fn})

Jika fungsi *loss* kustom Anda memiliki *hyperparameter* yang perlu disimpan, Anda bisa membuat *subclass* dari `tf.keras.losses.Loss` dan mengimplementasikan metode `get_config()`.

In [9]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

model.compile(loss=HuberLoss(2.0), optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2, validation_data=(X_valid, y_valid))

# Menyimpan model dengan class custom loss
# model.save("my_model_with_a_custom_loss_class.h5")

# Memuat model dengan class custom loss
# model = tf.keras.models.load_model("my_model_with_a_custom_loss_class.h5",
#                                    custom_objects={"HuberLoss": HuberLoss})

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0744 - val_loss: 0.0565
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0682 - val_loss: 0.0543


### 3.3. Fungsi Aktivasi Kustom, Inisialisasi, Regularizer, dan Batasan

Sebagian besar fungsionalitas Keras (misalnya, *losses*, *regularizers*, *constraints*, *initializers*, *metrics*, fungsi aktivasi, *layers*, dan bahkan model lengkap) dapat dikustomisasi dengan cara yang sama.  Seringkali, Anda hanya perlu menulis fungsi sederhana dengan *input* dan *output* yang sesuai.

In [10]:
def my_softplus(z): # Contoh fungsi aktivasi kustom
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32): # Contoh inisialisasi kustom
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights): # Contoh regularizer kustom
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # Contoh batasan (constraint) kustom
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

layer_custom = tf.keras.layers.Dense(30, activation=my_softplus,
                                     kernel_initializer=my_glorot_initializer,
                                     kernel_regularizer=my_l1_regularizer,
                                     kernel_constraint=my_positive_weights)

### 3.4. Metrik Kustom

*Loss* dan metrik secara konseptual berbeda: *loss* digunakan oleh *Gradient Descent* untuk melatih model dan harus *differentiable*, sementara metrik digunakan untuk mengevaluasi model dan harus mudah diinterpretasikan.

Metrik *streaming* (atau *stateful*) adalah metrik yang diperbarui secara bertahap, *batch* demi *batch*.  Anda dapat membuat *subclass* dari `tf.keras.metrics.Metric` untuk metrik *streaming*.

In [12]:
# Contoh data fiktif (seperti di notebook Anda)
X_train_scaled = np.random.rand(100, 10).astype(np.float32)
y_train = np.random.rand(100, 1).astype(np.float32)
X_valid = np.random.rand(20, 10).astype(np.float32)
y_valid = np.random.rand(20, 1).astype(np.float32)

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = huber_fn # Menggunakan fungsi huber_fn yang sudah didefinisikan
        # PERBAIKAN DI SINI: tambahkan argumen shape=() untuk variabel skalar
        self.total = self.add_weight(name="total", initializer="zeros", shape=())
        self.count = self.add_weight(name="count", initializer="zeros", shape=())

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# Uji model dengan metrik yang diperbaiki
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=[10]),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="nadam", metrics=[HuberMetric(2.0)])
history = model.fit(X_train_scaled, y_train, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - huber_metric_1: 0.2565 - loss: 0.5255 - val_huber_metric_1: 0.3239 - val_loss: 0.6791
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - huber_metric_1: 0.2315 - loss: 0.4715 - val_huber_metric_1: 0.2955 - val_loss: 0.6157


### 3.5. Lapisan Kustom

Jika Anda ingin membangun arsitektur yang berisi lapisan *exotic* yang tidak disediakan oleh TensorFlow, atau jika Anda ingin memperlakukan blok lapisan berulang sebagai satu lapisan, Anda dapat membuat lapisan kustom.

Untuk lapisan tanpa bobot, gunakan `tf.keras.layers.Lambda`.  Untuk lapisan *stateful* (dengan bobot), buat *subclass* dari `tf.keras.layers.Layer`.

In [14]:
# Contoh data fiktif (seperti di notebook Anda)
X_train_scaled = np.random.rand(100, 10).astype(np.float32)
y_train = np.random.rand(100, 1).astype(np.float32)
X_valid = np.random.rand(20, 10).astype(np.float32)
y_valid = np.random.rand(20, 1).astype(np.float32)

def my_softplus(z): # Contoh fungsi aktivasi kustom
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32): # Contoh inisialisasi kustom
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights): # Contoh regularizer kustom
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # Contoh batasan (constraint) kustom
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # Harus di akhir

    def call(self, X):
        return self.activation(tf.matmul(X, self.kernel) + self.bias)

    def compute_output_shape(self, batch_input_shape):
        # PERBAIKAN DI SINI: Akses elemen shape secara langsung atau konversi ke list yang benar
        # Cara paling robust adalah dengan mengubahnya menjadi list terlebih dahulu
        input_shape_list = list(batch_input_shape) # Mengkonversi tf.TensorShape menjadi list
        return tf.TensorShape(input_shape_list[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": tf.keras.activations.serialize(self.activation)}

# Menggunakan lapisan kustom
model_custom_layer = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[10]),
    MyDense(5, activation="relu"),
    tf.keras.layers.Dense(1)
])
model_custom_layer.compile(loss="mse", optimizer="sgd")
model_custom_layer.fit(X_train_scaled, y_train, epochs=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.2084


Jika lapisan Anda perlu memiliki perilaku yang berbeda selama pelatihan dan pengujian (misalnya, jika menggunakan `Dropout` atau `BatchNormalization`), Anda harus menambahkan argumen `training` ke metode `call()` dan menggunakannya untuk memutuskan apa yang harus dihitung.

### 3.6. Model Kustom

Seperti yang telah kita lihat di Bab 10, membuat kelas model kustom sangat mudah: cukup buat *subclass* dari kelas `tf.keras.Model`, buat *layer* dan *variabel* di *constructor*, dan implementasikan metode `call()` untuk melakukan apa pun yang Anda inginkan model lakukan.

In [15]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="elu",
                                            kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # Skip connection

class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation="elu",
                                            kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3): # Asumsi loop contoh dari buku
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

model_custom_residual = ResidualRegressor(output_dim=1)
model_custom_residual.compile(loss="mse", optimizer="sgd")
model_custom_residual.fit(X_train_scaled, y_train, epochs=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan    


Kelas `Model` adalah *subclass* dari kelas `Layer`, jadi model dapat didefinisikan dan digunakan persis seperti *layer*.  Namun, sebuah model memiliki fungsionalitas tambahan, seperti metode `compile()`, `fit()`, `evaluate()`, dan `predict()`.

### 3.7. *Loss* dan Metrik Berbasis Internal Model

Anda bisa mendefinisikan *loss* atau metrik berdasarkan bagian lain dari model Anda, seperti bobot atau aktivasi lapisan tersembunyi.  Ini bisa berguna untuk tujuan *regularization* atau untuk memantau aspek internal model.

Untuk mendefinisikan *loss* kustom berdasarkan internal model, hitung *loss* tersebut berdasarkan bagian model yang Anda inginkan, lalu berikan hasilnya ke metode `add_loss()`.

In [16]:
class ReconstructingRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(30, activation="selu",
                                            kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = tf.keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss) # Menambahkan loss kustom
        return self.out(Z)

model_reconstructing = ReconstructingRegressor(output_dim=1)
model_reconstructing.compile(loss="mse", optimizer="rmsprop")
model_reconstructing.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.6315
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1580 


### 3.8. Menghitung Gradien Menggunakan *Autodiff*

TensorFlow memungkinkan Anda menghitung gradien secara otomatis menggunakan *autodiff*.  Anda dapat menggunakan konteks `tf.GradientTape()` untuk merekam operasi yang melibatkan *variabel*, lalu meminta *tape* untuk menghitung gradien.

In [17]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)

def f(w1, w2):
    return 3 * w1**2 + 2 * w1 * w2

with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


*Tape* secara otomatis dihapus setelah Anda memanggil metode `gradient()`.  Jika Anda perlu memanggilnya lebih dari sekali, Anda harus membuat *tape* menjadi *persistent*.

In [18]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)
print(dz_dw1)
print(dz_dw2)
del tape # Penting untuk menghapus tape persistent

tf.Tensor(36.0, shape=(), dtype=float32)
tf.Tensor(10.0, shape=(), dtype=float32)


### 3.9. Loop Pelatihan Kustom

Dalam kasus yang jarang terjadi, metode `fit()` mungkin tidak cukup fleksibel untuk kebutuhan Anda.  Misalnya, jika Anda perlu menggunakan *optimizer* yang berbeda untuk bagian yang berbeda dari jaringan saraf Anda.

In [23]:
# Contoh sederhana loop pelatihan kustom (tanpa validasi atau metrik penuh)
# Untuk contoh lengkap, lihat buku atau notebook GitHub

l2_reg = tf.keras.regularizers.l2(0.05)
model_custom_loop = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                         kernel_regularizer=l2_reg),
    tf.keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mse

n_epochs = 5
batch_size = 32
n_steps_per_epoch = len(X_train_scaled) // batch_size

for epoch in range(n_epochs):
    print(f"Epoch {epoch + 1}/{n_epochs}")
    for step in range(n_steps_per_epoch):
        # Ambil batch acak (sederhana, untuk contoh)
        idx = np.random.randint(len(X_train_scaled), size=batch_size)
        X_batch, y_batch = X_train_scaled[idx], y_train[idx]

        with tf.GradientTape() as tape:
            y_pred = model_custom_loop(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model_custom_loop.losses) # Tambahkan losses regularisasi

        gradients = tape.gradient(loss, model_custom_loop.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_custom_loop.trainable_variables))

    # Evaliasi sederhana di akhir epoch
    val_loss = tf.reduce_mean(loss_fn(y_valid, model_custom_loop(X_valid)))
    print(f" - val_loss: {val_loss.numpy():.4f}")

Epoch 1/5
 - val_loss: 0.2183
Epoch 2/5
 - val_loss: 0.2116
Epoch 3/5
 - val_loss: 0.1838
Epoch 4/5
 - val_loss: 0.1792
Epoch 5/5
 - val_loss: 0.1245


Menulis *loop* pelatihan kustom memberi Anda kendali penuh, tetapi juga membuat kode lebih panjang dan lebih rentan terhadap *error*.

## 4. Fungsi dan *Graph* TensorFlow

TensorFlow dapat mengoptimalkan fungsi Python dengan mengubahnya menjadi *TensorFlow Functions* (*TF Functions*).  Ini biasanya mempercepat eksekusi, terutama untuk komputasi yang kompleks.

### 4.1. Fungsi dan *Concrete Functions*

*TF Functions* bersifat *polymorphic*, artinya mereka mendukung *input* dengan tipe dan *shape* yang berbeda.  Setiap kali Anda memanggil *TF Function* dengan kombinasi *input* baru, ia menghasilkan *concrete function* baru dengan *graph* sendiri yang khusus untuk kombinasi tersebut.

In [24]:
@tf.function
def tf_cube(x):
    print(f"x = {x}") # Ini hanya akan dieksekusi saat fungsi dilacak (traced)
    return x ** 3

# Panggilan pertama: melacak fungsi untuk int32 skalar
result_int = tf_cube(2)
print(result_int)

# Panggilan kedua: menggunakan fungsi yang dilacak
result_int_again = tf_cube(3)
print(result_int_again)

# Panggilan ketiga: melacak fungsi untuk float32 skalar
result_float = tf_cube(tf.constant(2.0))
print(result_float)

# Panggilan keempat: melacak fungsi untuk float32 tensor shape (1,)
result_tensor_1d = tf_cube(tf.constant([2.0]))
print(result_tensor_1d)

x = 2
tf.Tensor(8, shape=(), dtype=int32)
x = 3
tf.Tensor(27, shape=(), dtype=int32)
x = Tensor("x:0", shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
x = Tensor("x:0", shape=(1,), dtype=float32)
tf.Tensor([8.], shape=(1,), dtype=float32)


Anda dapat melihat *graph* komputasi dari *concrete function* menggunakan atribut `.graph`.

### 4.2. *AutoGraph* dan *Tracing*

TensorFlow menghasilkan *graph* melalui *AutoGraph* dan *tracing*. *AutoGraph* menganalisis kode sumber Python untuk menangkap semua pernyataan alur kontrol (seperti *loop* `for`, `while`, `if`).  Kemudian TensorFlow memanggil fungsi yang telah "ditingkatkan" ini dengan *symbolic tensor* (tanpa nilai sebenarnya, hanya nama, tipe data, dan *shape*).

*Loop* `for` yang berulang pada `tf.range()` akan diubah menjadi *dynamic loop* dalam *graph* komputasi, sedangkan *loop* `for` yang berulang pada `range()` Python biasa akan menjadi *static loop* dan "dilepaskan" selama *tracing*.

In [26]:
@tf.function
def add_10_dynamic(x):
    for i in tf.range(10): # Menggunakan tf.range untuk loop dinamis
        x += 1
    return x

# Melihat operasi graph untuk loop dinamis
# print(add_10_dynamic.get_concrete_function(tf.constant(0)).graph.get_operations())

@tf.function
def add_10_static(x):
    for i in range(10): # Menggunakan range() untuk loop statis
        x += 1
    return x

# Melihat operasi graph untuk loop statis (akan ada 10 operasi 'add')
print(add_10_static.get_concrete_function(tf.constant(0)).graph.get_operations())

[<tf.Operation 'x' type=Placeholder>, <tf.Operation 'add/y' type=Const>, <tf.Operation 'add' type=AddV2>, <tf.Operation 'add_1/y' type=Const>, <tf.Operation 'add_1' type=AddV2>, <tf.Operation 'add_2/y' type=Const>, <tf.Operation 'add_2' type=AddV2>, <tf.Operation 'add_3/y' type=Const>, <tf.Operation 'add_3' type=AddV2>, <tf.Operation 'add_4/y' type=Const>, <tf.Operation 'add_4' type=AddV2>, <tf.Operation 'add_5/y' type=Const>, <tf.Operation 'add_5' type=AddV2>, <tf.Operation 'add_6/y' type=Const>, <tf.Operation 'add_6' type=AddV2>, <tf.Operation 'add_7/y' type=Const>, <tf.Operation 'add_7' type=AddV2>, <tf.Operation 'add_8/y' type=Const>, <tf.Operation 'add_8' type=AddV2>, <tf.Operation 'add_9/y' type=Const>, <tf.Operation 'add_9' type=AddV2>, <tf.Operation 'Identity' type=Identity>]


### 4.3. Aturan Fungsi TF

Ada beberapa aturan yang perlu diperhatikan saat menulis *TF Functions*:
* Gunakan operasi TensorFlow, bukan pustaka eksternal (NumPy, dll.), karena panggilan ini hanya berjalan selama *tracing*.
* Hindari efek samping non-TensorFlow yang tidak terduga, karena efek samping ini hanya terjadi selama *tracing*.
* Buat *variabel* TensorFlow di luar *TF Function* atau pada panggilan pertama, dan gunakan metode `assign()` untuk memodifikasinya (bukan operator `=`, `+=`, dll.).
* Kode sumber fungsi Python Anda harus tersedia untuk TensorFlow.
* Gunakan `tf.range()` untuk *loop* yang ingin Anda sertakan dalam *graph* komputasi.

## Kesimpulan

Bab ini telah memberikan pemahaman yang mendalam tentang cara mengkustomisasi model dan alur pelatihan di TensorFlow, mulai dari tingkat API rendah hingga penggunaan *TF Functions* untuk optimasi. Dengan pengetahuan ini, Anda dapat membangun arsitektur jaringan saraf yang lebih kompleks dan menyesuaikan proses pelatihan untuk memenuhi kebutuhan spesifik proyek *Machine Learning* Anda.